
# Perspectief 1 – EV’s als primaire oplossing
Dit hoofdstuk onderzoekt of een stijgend aandeel elektrische voertuigen (EV’s) leidt tot **lagere CO₂‑uitstoot** van nieuw geregistreerde auto’s.  
We gebruiken twee visualisaties:

1. **Lijngrafiek** – EV‑aandeel per land (2010‑2023)  
2. **Scatter + slider** – EV‑aandeel vs. CO₂‑uitstoot per land/jaar


In [1]:

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# relatieve paden vanuit docs/
cars = pd.read_csv('../data/cars_by_fuel.csv')
co2  = pd.read_csv('../data/co2_new_cars.csv')

# ---------- Voorbewerking ----------
tmp = {}                       # key = geo-year
for _, r in cars.iterrows():
    key = f"{r.geo}-{r.TIME_PERIOD}"
    tmp.setdefault(key, {'geo': r.geo, 'year': int(r.TIME_PERIOD),
                         'elc': 0, 'total': 0})
    if r.mot_nrg == 'ELC':
        tmp[key]['elc'] = r.OBS_VALUE
    elif r.mot_nrg == 'TOTAL':
        tmp[key]['total'] = r.OBS_VALUE

for _, r in co2.iterrows():
    key = f"{r.geo}-{r.TIME_PERIOD}"
    if key in tmp:
        tmp[key]['co2'] = r.OBS_VALUE

df = (pd.DataFrame(tmp.values())
        .dropna(subset=['co2'])
        .assign(evShare=lambda d: d.elc / d.total * 100))

focus = df[df.geo.isin(['NL', 'DE', 'NO', 'PL', 'FR', 'ES'])]  # zes landen
code2name = dict(NL='Nederland', DE='Duitsland', NO='Noorwegen',
                 PL='Polen',     FR='Frankrijk', ES='Spanje')


## Visualisatie 1.1 – EV‑aandeel per land (2010‑2023)

In [2]:

fig_line = px.line(
    focus.replace({'geo': code2name}),
    x='year', y='evShare', color='geo',
    title='EV‑aandeel in geselecteerde landen',
    labels=dict(year='Jaar', evShare='EV‑aandeel (%)', geo='Land')
)
fig_line.update_traces(mode='lines+markers')
fig_line.update_yaxes(ticksuffix='%')
fig_line.show()


**Interpretatie**  
Noorwegen springt eruit met het hoogste EV‑aandeel, terwijl Polen en Spanje achterblijven. Beleidskeuzes en laadinfrastructuur blijken bepalend.

## Visualisatie 1.2 – CO₂‑uitstoot vs. EV‑aandeel (slider)

In [3]:

frames = []
for yr, grp in focus.groupby('year'):
    frames.append(go.Frame(
        name=str(yr),
        data=[go.Scatter(
            x=grp.evShare,
            y=grp.co2,
            mode='markers',
            text=grp.geo.map(code2name),
            marker=dict(size=10, color='#3b82f6', line=dict(width=1, color='#1e40af')),
            hovertemplate='%{text}<br>EV: %{x:.2f}%<br>CO₂: %{y:.1f} g/km<extra></extra>'
        )]
    ))

layout = go.Layout(
    title=f'CO₂‑uitstoot vs. EV‑aandeel ({frames[0].name})',
    xaxis=dict(title='EV‑aandeel (%)', ticksuffix='%'),
    yaxis=dict(title='Gem. CO₂ (g/km)'),
    sliders=[dict(
        steps=[dict(method='animate',
                    label=f.name,
                    args=[[f.name],
                          dict(mode='immediate',
                               frame=dict(duration=400, redraw=False),
                               transition=dict(duration=300))])
               for f in frames],
        currentvalue=dict(prefix='Jaar: ')
    )]
)

fig_scatter = go.Figure(data=frames[0].data, frames=frames, layout=layout)
fig_scatter.show()


Ook hier zien we een **negatieve correlatie**: landen met een groter EV‑aandeel hebben doorgaans lagere gemiddelde CO₂‑uitstoot per kilometer.